In [ ]:
%reload_ext autoreload
%autoreload 2

import neurite as ne
import numpy as np
import tensorflow.keras.backend as K
import tensorflow as tf
import nibabel as nib

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

def process_brats_vol(img_vol, pad=False, is_seg=False):
    img_vol = np.rot90(img_vol.transpose(2, 0, 1), axes=(1, 2), k=3)
    img_vol = img_vol[13:-14]
    
    if not is_seg:
        img_vol = img_vol / img_vol.mean()
        img_vol = np.interp(img_vol, (img_vol.min(), img_vol.max()), (0, 1))
    
    if pad:
        img_vol = np.pad(img_vol, pad_width=[(0, 0), (8, 8), (8, 8)], mode='constant', constant_values=0)
    return img_vol

In [ ]:
mi_loss = ne.tf.losses.MutualInformation()

In [ ]:
data = np.load('/home/srivathsa/projects/studies/gad/vmorph/data/brats/train/BraTS2021_00000.npz')['data']
img1 = tf.convert_to_tensor(data[0][..., None], dtype=tf.float32)
img2 = tf.convert_to_tensor(data[1][..., None], dtype=tf.float32)

print(img1.shape[-1])

mi_val = -mi_loss.volumes(img1, img2)

In [ ]:
v = K.eval(mi_val)

In [ ]:
v.mean()

In [ ]:
fpath_base = '/home/srivathsa/projects/brats_2021/data/val'
case_num = 'BraTS2021_00006'

t1_pre = process_brats_vol(nib.load('{}/{}/{}_t1.nii.gz'.format(fpath_base, case_num, case_num)).get_fdata(), pad=True)
t1_post = process_brats_vol(nib.load('{}/{}/{}_t1ce.nii.gz'.format(fpath_base, case_num, case_num)).get_fdata(), pad=True)

mi_val = -mi_loss.volumes(t1_pre.astype(np.float32), t1_post.astype(np.float32))
v = K.eval(mi_val)
print(v.mean())